<h1>Pyrororobocop</h1>

In [1]:
import pandas as pd
from analysis import recall_matrix

class Pyro(object):
    """
    Data object for the pyrec package

    This class contains free recall data and metadata that will be used by pyrec.

    Attributes
    ----------
    
    pres : pd.DataFrame
        Dataframe containing the presented words.  Each row represents the presented words for a given list and each column
        represents a list. The cells should be lowercase words. The index will be a multi-index, where the first level reprensents the subject number
        and the second level represents the list number
    
    features : pd.DataFrame
        Dataframe containing the features for presented words.  Each row represents the presented words for a given list and each column
        represents a list. The cells should be a dictionary of features, where the keys are the name of the features, and the values are the feature values.
        The index will be a multi-index, where the first level reprensents the subject number and the second level represents the list number
        
    rec : pd.DataFrame
        Dataframe containing the words recalled.  Each row represents the recalled words for a given list and each column
        represents a list.  Each row represents the recalled words for a given list and each column
        represents a list. The cells should be lowercase words. The index will be a multi-index, where the first level reprensents the subject number
        and the second level represents the list number
    
    meta : dict (optional)
        Meta data about the study (i.e. version, description, date, etc.) can be saved here
    
    """


    def __init__(self, pres=pd.DataFrame(), features=pd.DataFrame(), rec=pd.DataFrame(), meta={}):

        self.pres=pres
        self.features=features
        self.rec=rec
        self.meta=meta
        self._recall_mtx = recall_matrix(self.pres, self.rec)



In [2]:
Pyro()

In [3]:
from sqlalchemy import create_engine, MetaData, Table
import json
import pandas as pd
import numpy as np
import math
from __future__ import division
import re
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy.ma as ma
from itertools import izip_longest
from collections import Counter
from analysis import recall_matrix

/Users/kirstenziman/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


<h1> Load data in dataframe</h1>

In [4]:
db_url = "sqlite:///participants.db"
table_name = 'turkdemo'
data_column_name = 'datastring'

# boilerplace sqlalchemy setup
engine = create_engine(db_url)
metadata = MetaData()
metadata.bind = engine
table = Table(table_name, metadata, autoload=True)

# make a query and loop through
s = table.select()
rows = s.execute()

data = []
for row in rows:
    data.append(row[data_column_name])
    
# Now we have all participant datastrings in a list.
# Let's make it a bit easier to work with:

# parse each participant's datastring as json object
# and take the 'data' sub-object
data = [json.loads(part)['data'] for part in data if part is not None]

# insert uniqueid field into trialdata in case it wasn't added
# in experiment:
for part in data:
    for record in part:
#         print(record)
        if type(record['trialdata']) is list:

            record['trialdata'] = {record['trialdata'][0]:record['trialdata'][1]}
        record['trialdata']['uniqueid'] = record['uniqueid']
        
# flatten nested list so we just have a list of the trialdata recorded
# each time psiturk.recordTrialData(trialdata) was called.
def isNotNumber(s):
    try:
        float(s)
        return False
    except ValueError:
        return True

data = [record['trialdata'] for part in data for record in part]

# filter out fields that we dont want using isNotNumber function
filtered_data = [{k:v for (k,v) in part.items() if isNotNumber(k)} for part in data]
    
# Put all subjects' trial data into a dataframe object from the
# 'pandas' python library: one option among many for analysis
data_frame = pd.DataFrame(filtered_data)

<h1> Track Experiement Number</h1>

In [5]:
db_url = "sqlite:///participants.db"
table_name = 'turkdemo'
data_column_name = 'codeversion'

# boilerplace sqlalchemy setup
engine = create_engine(db_url)
metadata = MetaData()
metadata.bind = engine
table = Table(table_name, metadata, autoload=True)

# make a query and loop through
s = table.select()
rows = s.execute()

versions = []
for row in rows:
    versions.append(row[data_column_name])
    
version_col = []
for idx,sub in enumerate(data_frame['uniqueid'].unique()):
    for i in range(sum(data_frame['uniqueid']==sub)):
        version_col.append(versions[idx])
data_frame['exp_version']=version_col

<h1>Word Pool</h1>

In [6]:
# read in stimulus library
wordpool = pd.read_csv('cut_wordpool.csv')

<h1>Select Experiment</h1>

In [7]:
subids = list(data_frame[data_frame['listNumber']==15]['uniqueid'].unique())

d = dict()
for sub in subids:
    key = data_frame[data_frame['uniqueid']==sub]['exp_version'].values[0]
    if key in d:
        d[key].append(sub)
    else:
        d[key]=[sub]

#reaplce these values with the experiment number
#three values for the case of experiment 1 only
#print (d["0.0"], d['1.0'], d['1.1'])

exp1=d["0.0"]+d['1.0']+d['1.1']


exp1.remove('debugGPNALW:debugXSJ1FD')
exp1.remove('debugLXMXTP:debugJAXRZL')
exp1.remove('debugHP65NS:debugLWS9KB')
exp1.remove('debugKUWU41:debug9FG9EP')


# subids.remove('debug4PXFJG:debug3V9BT9')
# subids.remove('debugAD2211:debugB3TKJQ')
# subids.remove('debug7XDZDR:debugO8OCCV') # all of the audio files are empty ?!
# subids.remove('debugTX7U35:debugZFTPLT')
##################

print(exp1)

[u'debugN8TPWO:debugF1XWCH', u'debugZQ55YL:debug5WQHPC', u'debugWF2JFB:debugPNRZFQ', u'debugQRX0V3:debugFIWAG8', u'debugIAU8V9:debugT1DECK', u'debug02E4FI:debugF7UOXH', u'debugS4GATI:debug2LRP6X', u'debugJAPX2W:debugFZOLSG', u'debugA98B98:debug5H8QRL', u'debugKDM8HT:debugH2I05W', u'debugQS9870:debugKM1SRC', u'debugVFPD79:debugIP75FV', u'debugX84L2K:debugCDN40O', u'debugSU1T93:debugKCB9VM', u'debugE1CAO3:debugONZ2R5', u'debug8DEMRS:debugC55CO6', u'debugXUZA8U:debugMR3K3X']


<h1>Processing Functions</h1>

In [8]:
# this function takes the data frame and returns subject specific data based on the subid variable
def filterData(data_frame,subid):
    filtered_stim_data = data_frame[data_frame['stimulus'].notnull() & data_frame['listNumber'].notnull()]
    filtered_stim_data = filtered_stim_data[filtered_stim_data['trial_type']=='single-stim']
    filtered_stim_data =  filtered_stim_data[filtered_stim_data['uniqueid']==subid]
    return filtered_stim_data

# this function parses the data creating an array of dictionaries, where each dictionary represents a trial (word presented) along with the stimulus attributes
def createStimDict(data):
    stimDict = []
    for index, row in data.iterrows():
        stimDict.append({
                'text': str(re.findall('>(.+)<',row['stimulus'])[0]),
                'color' : { 'r' : int(re.findall('rgb\((.+)\)',row['stimulus'])[0].split(',')[0]),
                           'g' : int(re.findall('rgb\((.+)\)',row['stimulus'])[0].split(',')[1]),
                           'b' : int(re.findall('rgb\((.+)\)',row['stimulus'])[0].split(',')[2])
                           },
                'location' : {
                    'top': float(re.findall('top:(.+)\%;', row['stimulus'])[0]),
                    'left' : float(re.findall('left:(.+)\%', row['stimulus'])[0])
                    },
                'category' : wordpool['CATEGORY'].iloc[list(wordpool['WORD'].values).index(str(re.findall('>(.+)<',row['stimulus'])[0]))],
                'size' : wordpool['SIZE'].iloc[list(wordpool['WORD'].values).index(str(re.findall('>(.+)<',row['stimulus'])[0]))],
                'wordLength' : len(str(re.findall('>(.+)<',row['stimulus'])[0])),
                'firstLetter' : str(re.findall('>(.+)<',row['stimulus'])[0])[0],
                'listnum' : row['listNumber']
            })
    return stimDict

# this function loads in the recall data into an array of arrays, where each array represents a list of words
def loadRecallData(subid):
    recalledWords = []
    for i in range(0,16):
        try:
            f = open('recall_data/' + subid + '-' + str(i) + '.wav.txt', 'rb')
            try:
                spamreader = csv.reader(f, delimiter=' ', quotechar='|')
            except:
                f = open('recall_data/' + subid + '/' + subid + '-' + str(i) + '.wav.txt', 'rb')
                spamreader = csv.reader(f, delimiter=' ', quotechar='|')
        except (IOError, OSError) as e:
            print(e)
        for row in spamreader:
            recalledWords.append(row[0].split(','))
    return recalledWords

# this function computes accuracy for a series of lists
def computeListAcc(stimDict,recalledWords):
    accVec = []
    for i in range(0,16):
        stim = [stim['text'] for stim in stimDict if stim['listnum']==i]
        recalled= recalledWords[i]
        
        acc = 0
        tmpstim = stim[:]
        for word in recalled:
            if word in tmpstim:
                tmpstim.remove(word)
                acc+=1
        accVec.append(acc/len(stim))
    return accVec

<h1>Create presentedWords and recalledWords</h1>

In [9]:
experiment=[]
all_recalled=[]
all_presented=[]
for idx,sub in enumerate(exp1):

    filteredStimData = filterData(data_frame,sub)
    stimDict = createStimDict(filteredStimData)
    #get and parse subject's data

    presentedWords = [[] for i in range(0,16)]  
    for data in stimDict:
        presentedWords[int(data['listnum'])].append(data['text'])

    recalledWords = loadRecallData(sub)
    experiment.append(recall_matrix(presentedWords, recalledWords))
    
    all_recalled.append(recalledWords)
    all_presented.append(presentedWords)
    
print(len(all_presented))
print(all_presented)

17
[[['OAK', 'JUMPER', 'MARIMBA', 'ROBE', 'COLANDER', 'BLENDER', 'COTTONWOOD', 'EVERGREEN', 'HARMONICA', 'BANJO', 'BELT', 'UKULELE', 'SPRUCE', 'CUP', 'SHIRT', 'MIXER'], ['SAW', 'CLARINET', 'PIANO', 'KAZOO', 'FERRET', 'CHISEL', 'SQUIRREL', 'ENGLAND', 'DRUM', 'LEOPARD', 'CAT', 'BOLTS', 'FRANCE', 'JAPAN', 'PROTRACTOR', 'SWITZERLAND'], ['LOG', 'BROCCOLI', 'CARPET', 'PICKLE', 'GRIDDLE', 'DISH', 'AZALEA', 'STRAINER', 'BUTTERCUP', 'SAUCER', 'FOUNDATION', 'FLOOR', 'ONION', 'ELEVATOR', 'CAULIFLOWER', 'DAHLIA'], ['WASHER', 'WISCONSIN', 'MANGO', 'VANCOUVER', 'VERMONT', 'HAWAII', 'TOKYO', 'JERUSALEM', 'KUMQUAT', 'RULER', 'PHILADELPHIA', 'NECTARINE', 'CALIFORNIA', 'GLUE', 'BANANA', 'NUTS'], ['SCREWS', 'HONEYDEW', 'PINE', 'ELM', 'RASPBERRY', 'HAMMER', 'FIG', 'HICKORY', 'SAXOPHONE', 'PLUM', 'WIRE', 'TRIANGLE', 'MAPLE', 'WEDGE', 'VIOLIN', 'CELLO'], ['LEMON', 'IOWA', 'MICHIGAN', 'POMEGRANATE', 'KANSAS', 'RIB', 'MONKEY', 'RACOON', 'CLEMENTINE', 'MOOSE', 'FLORIDA', 'TONGUE', 'TOOTH', 'PEAR', 'ANTELOPE', 

<h1>Create Presented & Recalled DataFrames</h1>

In [10]:
def list2pd(all_data):
    
    def make_multi_index(sub_data, isub):
        return pd.MultiIndex.from_tuples([(sub_num,lst_num) for lst_num,lst in enumerate(sub_data)], names = ['Subject', 'List'])
    
    subs_list_of_dfs = [pd.DataFrame(sub_data, index=make_multi_index(sub_data, sub_num)) for sub_num,sub_data in enumerate(all_data)]
    return pd.concat(subs_list_of_dfs)

list2pd(all_presented)

0             1             2            3   \
Subject List                                                         
0       0             OAK        JUMPER       MARIMBA         ROBE   
        1             SAW      CLARINET         PIANO        KAZOO   
        2             LOG      BROCCOLI        CARPET       PICKLE   
        3          WASHER     WISCONSIN         MANGO    VANCOUVER   
        4          SCREWS      HONEYDEW          PINE          ELM   
        5           LEMON          IOWA      MICHIGAN  POMEGRANATE   
        6          POLAND          YAMS      ETHIOPIA        ZEBRA   
        7          GLOVES        MANTIS     MILLIPEDE     SYCAMORE   
        8          WILLOW    EUCALYPTUS        BONGOS       GUITAR   
        9       NARCISSUS         KNIFE     SUNFLOWER        DRESS   
        10            MUG          ARMS           HIP      BROILER   
        11           OKRA       PUMPKIN  PENNSYLVANIA         OHIO   
        12         KIDNEY          FOOT        SQUASH    STOCKHOLM   
        13        SPINACH      CAMISOLE          IRAN       BLOUSE   
        14          GUAVA       GIRAFFE         HORSE          COW   
        15        KNUCKLE       KITCHEN  HIPPOPOTAMUS        HEART   
1       0          DALLAS          OVEN       STOMACH  THERMOMETER   
        1        ANTELOPE         LEMON    CLEMENTINE         IOWA   
        2         TERMITE         TEXAS     CHRYSALIS       GINGER   
        3       EVERGREEN         SHIRT       BLENDER    HARMONICA   
        4          FRANCE          DRUM       ENGLAND      LEOPARD   
        5            FOOT          WOLF     AUBERGINE        OLIVE   
        6           MAINE          LILY         DAISY        KNIFE   
        7     CAULIFLOWER        DAHLIA     BUTTERCUP       SAUCER   
        8            PINE          PLUM         CELLO       HAMMER   
        9           MOUSE       GIRAFFE         GLASS          POT   
        10       SYCAMORE      TROUSERS          RENO         PALM   
        11      NECTARINE       VERMONT        HAWAII        TOKYO   
        12         FINGER        BONGOS        POPLAR      MONARCH   
        13         POTATO        JACKET       CHIMNEY         IRAN   
...                   ...           ...           ...          ...   
15      2             PIG     AUSTRALIA         ZEBRA         BASS   
        3             CUP        JUMPER         BANJO         BELT   
        4        CORRIDOR           POT         HORSE          COW   
        5         PUMPKIN           FOX        GINGER       RABBIT   
        6         STOMACH           HIP   THERMOMETER         TUBA   
        7          ALCOVE  HIPPOPOTAMUS       PETUNIA         FACE   
        8           EGYPT          SUIT        JACKET      SPINACH   
        9         MONTANA     SUNFLOWER     NARCISSUS        DAISY   
        10         MANTIS      MOSQUITO          PALM      SWEATER   
        11        VERMONT        BANANA         RULER   CALIFORNIA   
        12         RACOON      ANTELOPE        KANSAS       TONGUE   
        13    SWITZERLAND        FERRET      SQUIRREL   PROTRACTOR   
        14        PANTHER        SQUASH     STOCKHOLM    AUBERGINE   
        15         AZALEA    FOUNDATION   CAULIFLOWER     BROCCOLI   
16      0           PARIS           MUG          TUBA      BROILER   
        1        ANTELOPE          IOWA   POMEGRANATE       KANSAS   
        2         TERMITE  PENNSYLVANIA          OKRA         OHIO   
        3          AZALEA        PICKLE    FOUNDATION        ONION   
        4            BASS        CARROT       MARACAS     ETHIOPIA   
        5           ANKLE        FINGER        GUITAR        LARVA   
        6       MILLIPEDE          PALM      SYCAMORE        CEDAR   
        7            GOAT     AUBERGINE         MIAMI        ELBOW   
        8       EVERGREEN    COTTONWOOD         BANJO       SPRUCE   
        9         PETUNIA        ALCOVE          ROSE      KITCHEN   
        10       TRIANGLE      HO

<h1>Create Recall Matrix Dataframe</h1>

In [48]:
def lists2rec_pd(all_presented, all_recalled):
    
    def make_multi_index(sub_data, isub):
        return pd.MultiIndex.from_tuples([(sub_num,lst_num) for lst_num,lst in enumerate(sub_data)], names = ['Subject', 'List'])
    
    def make_rec_mat(all_presented, all_recalled):
        all_recall_mats=[]
        if len(all_presented) == len(all_recalled):
            for x in range(0, len(all_presented)):
                rec_mat = recall_matrix(all_presented[x], all_recalled[x])
                all_recall_mats.append(rec_mat)
        
        return(all_recall_mats)
    
    rec_data = make_rec_mat(all_presented, all_recalled)
    subs_list_of_recs = [pd.DataFrame(sub_data, index=make_multi_index(sub_data, sub_num)) for sub_num,sub_data in enumerate(rec_data)]
    
    return(pd.concat(subs_list_of_recs))

In [49]:
lists2rec_pd(all_presented, all_recalled)

0     1     2     3     4     5     6     7     8     9  ...  \
Subject List                                                             ...   
0       0      1.0   2.0   3.0   9.0  14.0  15.0   NaN   NaN   NaN   NaN ...   
        1      1.0   6.0   3.0   2.0   8.0  12.0   NaN   NaN   NaN   NaN ...   
        2      2.0  15.0   8.0   3.0   1.0  -1.0   NaN   9.0   NaN   NaN ...   
        3      2.0   5.0   8.0   7.0   6.0  14.0  -2.0   NaN   9.0  15.0 ...   
        4      1.0   2.0  13.0   7.0   3.0   9.0   NaN   NaN   NaN   NaN ...   
        5      2.0   3.0   5.0   6.0   7.0   1.0  10.0  11.0   NaN   NaN ...   
        6      5.0   2.0   3.0  14.0   NaN  13.0   NaN   NaN   NaN   NaN ...   
        7      1.0   2.0   3.0  -6.0   NaN  10.0   5.0   3.0   NaN   NaN ...   
        8      1.0   2.0   3.0   4.0   5.0   7.0  16.0   NaN   NaN   NaN ...   
        9      9.0  10.0  14.0  15.0  12.0  -1.0   NaN  -6.0   NaN   NaN ...   
        10   -10.0   NaN   NaN  16.0   5.0   NaN   NaN   9.0  -1.0   NaN ...   
        11   -11.0   NaN  15.0   7.0  16.0  14.0   NaN   NaN   NaN   NaN ...   
        12    12.0 -10.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN ...   
        13     3.0   5.0  11.0   6.0  -5.0   NaN  -1.0   NaN   NaN   NaN ...   
        14     8.0   9.0  10.0  12.0  14.0   NaN   NaN   NaN   NaN   NaN ...   
        15     9.0   3.0 -10.0   NaN  -2.0   NaN   5.0   8.0   NaN   NaN ...   
1       0      1.0   6.0  10.0  15.0  16.0  12.0   NaN   5.0  11.0   NaN ...   
        1     12.0  16.0  11.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN ...   
        2      2.0   NaN  15.0   8.0  11.0  16.0   NaN   NaN   NaN   NaN ...   
        3      1.0   3.0   5.0  13.0  14.0  15.0  10.0   NaN   NaN   NaN ...   
        4      1.0   3.0  15.0   9.0  11.0  14.0   6.0   NaN   NaN   NaN ...   
        5     13.0  10.0  16.0  15.0   NaN   NaN   NaN   NaN   NaN   NaN ...   
        6      3.0   6.0   5.0   7.0   1.0   8.0  12.0  15.0   NaN   NaN ...   
        7      NaN   6.0   8.0  15.0   4.0   NaN   NaN   NaN   NaN   NaN ...   
        8      6.0   9.0  15.0  14.0  16.0   NaN   NaN   NaN   NaN   NaN ...   
        9      2.0   6.0   7.0   4.0  10.0  11.0  14.0  15.0  16.0   NaN ...   
        10    -9.0   NaN -10.0   NaN   6.0   9.0   NaN   NaN   NaN   NaN ...   
        11     3.0   5.0   4.0  10.0  11.0   6.0  13.0   NaN   NaN   NaN ...   
        12     1.0   5.0   9.0   6.0   4.0  14.0  13.0   NaN   NaN   NaN ...   
        13     7.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN ...   
...            ...   ...   ...   ...   ...   ...   ...   ...   ...   ... ...   
15      2      1.0   3.0  15.0  11.0  16.0   5.0   2.0   4.0   NaN   NaN ...   
        3     15.0   NaN   5.0  13.0   4.0   2.0   NaN   NaN   NaN   NaN ...   
        4     12.0  14.0   1.0   7.0  15.0   4.0   3.0  10.0   6.0  11.0 ...   
        5     13.0  14.0   5.0  16.0   9.0   NaN   NaN   NaN   NaN   NaN ...   
        6      7.0  16.0  12.0   9.0   NaN   NaN   NaN   NaN   NaN   NaN ...   
        7     15.0  16.0  14.0   2.0   3.0   7.0   8.0   NaN   NaN   NaN ...   
        8      1.0   8.0  14.0   5.0  10.0   4.0   3.0   NaN   NaN   NaN ...   
        9     12.0  16.0   2.0   7.0   6.0   8.0   NaN   NaN   NaN   NaN ...   
        10    16.0  11.0  15.0   3.0   1.0   2.0   9.0  14.0  10.0   NaN ...   
        11    15.0   1.0   4.0  13.0  11.0   8.0   9.0   NaN   NaN   NaN ...   
        12    16.0   8.0   2.0  14.0   NaN   NaN   NaN   NaN   NaN   NaN ...   
        13    16.0  13.0  10.0   5.0   NaN   NaN   NaN   NaN   NaN   NaN ...   
        14    15.0  16.0   5.0   2.0   1.0  14.0   NaN   NaN   NaN   NaN ...   
        15    16.0   1.0   6.0   1.0   NaN   NaN   NaN   NaN   NaN   NaN ...   
16      0     16.0   3.0  13.0   6.0  14.0   9.0   1.0   5.0   2.0   7.0 ...   
        1      NaN   NaN   NaN   7.0   6.0   9.0  10.0   NaN   NaN   NaN ...   
        2     16.0  10.0   3.0  12.0   2.0   7.0   6.0  16.0   NaN   NaN ...   
        3      7